In [1]:
from IPython.core.display import HTML, Image, Markdown
import sys
import json
sys.path.append("microTC")
from microtc.textmodel import TextModel
from itertools import combinations


from graphviz import Digraph
dot = Digraph(comment='microtc pipeline', format="png")
# dot.engine = 'circo'
dot.graph_attr['rankdir'] = 'LR'
dot.node('i', '', style="invis")
dot.node('n', 'Normalización')
dot.node('t', 'Tokenización')
dot.node('w', 'Pesado')
dot.node('c', 'Clasificación')
dot.node('o', '', style="invis")

dot.edge('i', 'n', label="texto entrada")
dot.edge('n', 't', label="texto normalizado")
dot.edge('t', 'w', label="bolsa de palabras")
dot.edge('w', 'c', label="vector con pesos")
dot.edge('c', 'o', label="clase")

pipeline = dot.render("fig-pipeline", view=False)


# Clasificación de texto #
## Un enfoque basado en $\mu TC$ ##


**Seminario de la Sociedad Matemática Mexicana SMM 2016**


<div>
    Eric Sadit Téllez Avila INFOTEC 
    <estellezav@conacyt.mx> <br/>
       CONACyT -- INFOTEC
</div>

# Agenda #
- ¿Qué es $\mu TC$
- ¿En qué consiste la tarea de clasificación de texto?
- ¿Cómo esta compuesto $\mu TC$?
- Estado del arte
- Cómo se compara $\mu TC$ con el estado del arte
- Qué falta en $\mu TC$
- Ejemplos


## Categorización de texto ##
El problema consiste en, dado un texto $d$, determinar la(s) categoría(s) a la que pertenece en un conjunto $C$ de categorias, previamente conocido.

Más formalmente:

Dado un conjunto de categorias $\cal{C} = \{c_1, ..., c_m\}$, determinar el subconjunto de categorias
$C_d \in \wp(\cal{C})$ a las que pertenece $d$. 

Notese que $C_t$ puede ser vacio o $\cal{C}$.



## Clasificación de texto ##
La _clasificación_ de texto es una especialización del problema de categorización, donde $|C_d| = 1$, esto es $d$ solo puede ser asignado a una categoría.

Es un problema de interés en la industria y la acádemia, con aplicaciones variadas a distintas áreas del conocimiento.

- Análisis de sentimiento
- Determinación de autoría, e.g., género, edad, estilo, etc.
- Detección de spam
- Categorización de noticias
- Clasificación de idioma

# Procesamiento de Lenguaje Natural #


Un documento $d=s_1\cdots s_n$ es simplemente la concatenación de símbolos $s \in \Sigma$. Donde, $\Sigma$ es un _alfabeto_ de tamaño $\sigma = |\Sigma|$

Notese qué el número de textos posibles de tamaño $n$ es $\sigma^n$, por ejemplo, limitados a texto en inglés en Twitter se tienen
    $$ 26^{140} \simeq 1.248 \times 10^{198} $$

Sin emabargo, en lenguaje natural, este número no suele ser tan grande:
  - existen reglas sobre que símbolos se pueden unir
  - más aún, hay noción de _terminos_ o _palabras_, i.e., _morfología_
  - también, hay reglas sobre como las palabras se pueden combinar, i.e., _sintaxis y gramática_

Sin embargo, es un problema sumamente complicado, hay muchas reglas, variantes, excepciones, errores, etc.

Y por si fuera poco, aunque los conceptos existen en esencia, aparecen de manera diferente en todos los lenguajes


Además, esta el problema semántico:

- un término $s_i$ tiene significados diferentes (antónimos)
- lo contrario también existe, $s_i \not= s_j$ pero que son idénticos en significado (sinónimos)
- en ambos casos, el significado preciso depende del contexto
- también hay casos _aproximados_ de todo lo anterior
- ironias, sarcamos, etc.

... hay muchísimos problemas abiertos. **NLP** es complicado, de hecho es _AI-complete_


# Nuestro Enfoque #
Por su complejidad, trabajar en NLP tiene una gran cantidad de problemas abiertos, en particular nosotros nos enfocamos en la clasificación de texto escrito de manera informal (e.g., Twitter).

Para esto se utiliza un _pipeline_ estándar

![Pipeline](fig-pipeline.png)

No es la única opción, pero fijar el pipeline es útil como ya se expondrá.


El enfoque teórico suele ser muy complicado, y en realidad poco efectivo en la práctica, dadas las simplificaciones necesarias para hacerlo manejable

- Lógica
- Lingüistica
- Semántica

El enfoque práctico supone muchas cosas, en particular es un tanto casuístico:
    
- Se fija el lenguaje
- Se fija el problema
- Y la raíz de todos los males, muchas veces se supone que entre más técnicas sofísticadas se usen, mejores resultados se tendrán

En ambos enfoques se suele suponer que ausencia de errores de diferentes fuentes, sin embargo, es la regla cuando el texto que se analiza fue generado por usuarios de una red social, e.g. Twitter.

# ¿Qué es $\mu TC$? #
micro TC o $\mu TC$ es un clasificador de texto desarrollado en
el _Laboratorio de Análisis Computacional de Grandes Cúmulos de Información_
(o _Laboratorio de BigDATA_) de INFOTEC, sede Aguascalientes.

Esta disponible para ser clonado en [https://github.com/INGEOTEC/microTC](https://github.com/INGEOTEC/microTC). Esta escrito en Python 3.5 para sacar ventaja de unicode. También se puede instalar utilizando `pip` y `conda`.


En particular, nuestro enfoque se basa en _aprendizaje computacional_ y _optimización combinatoria_. Hemos probado que este esquema es muy competitivo en la práctica. Además, con la adecuada selección de las funciones podemos lograr que 
$\mu TC$ se independiente del lenguaje y robusto a errores.

Esta compuesto por:
- una serie de funciones de transformación de texto
- una serie de tokenizadores
- filtros de palabras y
- algoritmos de pesado de términos

Todo esto orquestado mediante un algoritmo de optimización combinatoria


## Lista de parametros ##

### Normalizadores multilenguaje ###

|   nombre  | valores             |        descripción                   |
|-----------|---------------------|--------------------------------------|
|	del-punc | yes, no | Determina si las puntuaciones deben removerse |
|	del-d1   | yes, no | Determina si se deben borrar letras repetidas |
|	del-diac | yes, no | Determina si los simbolos que no ocupan espacios deben ser removidos |
|	lc       | yes, no | Determina si los símbolos deben ser normalizados en minúsculas |
|	emo      | remove, group, none | Controla como deben tratarse los emoticones |
|	num      | remove, group, none | `........................` números |
|	url      | remove, group, none | `........................` urls |
|	usr      | remove, group, none | `........................` usuarios |

configuraciones: 1296



### Normalizadores dependientes del lenguaje ###

|   nombre  | valores             |        descripción                   |
|-----------|---------------------|--------------------------------------|
|	stem   | yes, no | Determina si a las palabras se les aplica _stemming_. |
|	neg    | yes, no | Determina si los operadores de negación son manejados de manera especial |
|	sw | remove, group, none | Controla como los _stopwords_ son manejados |

configuraciones: 12


### Tokenizadores ###
Los tokenizadores son en realidad una lista de tokenizadores, y están definidos tokenizer un elemento en $\wp{(\text{n-words} \cup \text{q-grams} \cup \text{skip-grams})} \setminus \{\emptyset\}$

|   nombre  | valores             |        descripción                   |
|-----------|---------------------|--------------------------------------|
|	n-words    | $\{1,2,3\}$      | Longitud de n-gramas de palabras (n-words) |
|	q-grams  | $\{1,2,3,4,5,6,7\}$ | Longitud de q-gramas de caracteres) |
|	skip-grams  | $\{(2,1), (3, 1), (2, 2), (3, 2)\}$ | Lista de skip-grams|

configuraciones: 16383

### Parametros para pesado ###
|   nombre  | valores             |        descripción                   |
|-----------|---------------------|--------------------------------------|
|token_min_filter | $\{0.01, 0.03, 0.1, 0.30, -1, -5, -10\}$ | Filtro de frequencias bajas |
|token_max_filter | $\{0.9, 99, 1.0\}$ | Filtro de frequencias altas |
|	tfidf    | yes, no | Determina si se debe realizar un pesado TFIDF de terminos |

configuraciones = 42

In [2]:
conf = 42 * 16383 * 12 * 1292
time = conf * 10 / 60 / 24 / 365.25
print(conf, time)

10668085344 202830.73511293635


Con esto tenemos un total de $10,668,085,344$ configuraciones. Dependiendo del tamaño de cada colección, cada configuración se evalua en tiempo diferente.

Una tarea típica de análisis de sentimientos tiene un costo por configuración de cerca de 10 min. en una computadora de relativamente nueva, i.e., Intel(R) Xeon(R) CPU E5-2640 v3 @ 2.60GHz.

Esto da un total de $202,830.74$ años de tiempo de cómputo.

Un enfoque _naïve_ requiere una cantidad enorme de computadoras para parallelizar y distribuir este proceso, por esta razón, es mejor utilizar algoritmos eficientes para optimizar la búsqueda de la mejor configuración


In [3]:
from sympy import symbols, sin, cos, exp, tan, sqrt
from sympy.plotting import plot, plot3d
from matplotlib import pyplot
x, y = symbols('x y')
plot(-x**2+200, axis=False, show=False).save("opt-1a.png")
plot(sin(x)*sin(1/(x*x)), axis=False, show=False).save('opt-1b.png')
plot3d((-x**2 - y**2, (x, -5, 5), (y, -5, 5)), axis=False, show=False).save('opt-2a.png')
plot3d(sin(x*y), axis=False, show=False).save('opt-2b.png')

# Optimización combinatoria #
Para tener algo práctico utilizamos una aproximación a encontrar la configuración óptima, en a lo más unas cuantas horas.

Para esto necesitamos un espacio de búsqueda (dominio) y una función de aptitud.

El espacio de búsqueda son todas las configuraciones.

La función de aptitud es una médida de _que tan atinado_ es nuestro clasificador. Deseamos seleccionar la configuración que maximice la función de aptitud.



Notesé que:

- El espacio de configuración no esta en $\mathbb{R}^n$
- En la configuración/parámetros son una mezcla de reales y categóricos
- No hay coordenadas como tal

Afortunadamente

- Se pueden definir vecindades entre configuraciones
- Se puede construir una gráfica
- Se puede samplear el espacio
- Se puede definir una función de aptitud de nodos


## Maximizando un objetivo (ejemplo) ##

|fácil|un poco más complicado|
|-|-|
|<img src="opt-1a.png" style="width:90%" />|<img src="opt-1b.png" style="width:90%" />|
|<img src="opt-2a.png" style="width:90%" />|<img src="opt-2b.png" style="width:90%" />|


# Resultados Experimentales #
![datasets](descripcion-datasets.png)

![performance](performance-competencias.png)

![performance 2](performance2.png)

![performance 3](performance3.png)

# Gracias por su atención #

## ¿Comentarios y preguntas? ##


- Para más información sobre el _Posgrado en Ciencias en Ciencia de Datos_ aquí en Aguascalientes: <eric.tellez@infotec.mx>
- Esta presentación la puedes obtener en: <https://github.com/INGEOTEC/pres-MSA-SMM2016>
- $\mu TC$ lo puedes conseguir en: <https://github.com/INGEOTEC/microTC>
- Página de INFOTEC <https://www.infotec.mx/>

# Slides de soporte #

## Sobre el pesado ##
El pesado de tokens esta fijo a TFIDF. Su nombre viene de la formulación $tf \times idf$

$tf$ es _term frequency_; es una medida de importancia **local** del término $t$ en el documento $d$, de manera normalizada esta definida como:
    $$tf(t,d) = \frac{freq(t, d)}{\max_{w \in d}{freq(w, d)}}$$
entre más veces aparece en el documento $d$, $t$ es más importante

$idf$ quiere decir _inverse document frequency_; es una medida **global** a la colección $D$, esta definida como:
$$ idf(t,d) = log{\frac{|D|}{1+|{d \in D: t \in d}|}} $$
entre más veces aparece $t$ en la colección, el término es más común y menos discriminante; por lo tanto, menos importante

## Sobre el clasificador ##
El clasificador es un algoritmo de aprendizaje computacional que dado un objeto, decide finalmente la etiqueta o clase de ese objeto. Tiene dos etapas bien definidas

- **Entrenamiento.** Dado un conjunto de ejemplos en un espacio vectorial, con etiquetas, el algoritmo intenta _aprender_ las características que definen cada clase
- **Predicción.** La idea es que una vez entrenado, el algoritmo puede recibir objetos no vistos durante la etapa de entrenamiento y asignales la clase adecuada

En particular, esta fijo como un _Support Vector Machine_ (SVM) con kernel lineal




# ¿Por qué es robusto a errores?

Considere los siguientes textos $T=I\_like\_vanilla$, $T' = I\_lik3\_vanila$

Para fijar ideas pongamos que se usar el coeficiente de Jaccard como medida de similitud, i.e.


$$\frac{|\{{I, like, vanilla}\} \cap \{{I, lik3, vanila}\}|}{|\{{I, like, vanilla}\} \cup \{{I, lik3, vanila}\}|} = 0.2$$


$$Q^T_3 = \{ I\_l, \_li, lik, ike, ke\_, e\_v, \_va, van, ani, nil, ill, lla \}$$
$$Q^{T'}_3 = \{ I\_l, \_li, lik, ik3, k3\_, 3\_v, \_va, van, ani, nil, ila \}$$

Bajo la misma medida
$$\frac{|Q^T_3 \cap Q^{T'}_3|}{|Q^T_3 \cup Q^{T'}_3|} = 0.448.$$

Se puede ver que estos conjuntos son más similares que los tokenizados por palabra

La idea es que un algoritmo de aprendizaje tenga un poco más de soporte para determinar que $T$ es parecido a $T'$


In [4]:
def emoformat(A, emo):
    s = "  ".join([a[0] for a in A if a[1] == emo])
    return s[:1000] + "..."
    
with open('microTC/microtc/resources/emoticons.json') as f:
    A = list(json.load(f).items())
    A.sort()
    S = dict(
        pos=emoformat(A, '_pos'),
        neg=emoformat(A, '_neg'),
        neu=emoformat(A, '_neu'),
        none=emoformat(A, '_none'),
    )

output = ["## Emoticones y emojis clasificados por sentimiento ##"]
for k, v in S.items():
    output.append("## Clase `{0}` ##".format(k.upper()))
    output.append(v)

Markdown("\n".join(output))

## Emoticones y emojis clasificados por sentimiento ##
## Clase `NONE` ##
'~'U  ^( \\\'-\\\' )^  π  ϟ  ಠ~ಠ  ‷̗ↂ凸ↂ‴  ‼  ⁂  ⁉  ⁎  ⁑  ℹ  ↶  ↷  ∨  ∩  ∪  ∴  ⊕  ⊗  ⊚  ⊛  ⊜  ⊥  ⌆  ⌘  ⌥  ⌫  ⎈  ⎋  ⎌  ⏎  ⏏  ╽  ╿  ◉  ●  ◐  ◑  ◒  ◓  ◔  ◕  ◖  ◗  ☄  ☇  ☈  ☉  ☊  ☌  ☍  ☎  ☏  ☛  ☝  ☞  ☤  ☧  ☨  ☩  ☪  ☫  ☬  ☭  ☸  ☿  ♊  ♋  ♌  ♍  ♎  ♏  ♐  ♑  ♒  ♓  ♤  ♨  ♭  ♮  ♯  ♿  ⚒  ⚓  ⚔  ⚕  ⚘  ⚚  ⚜  ⚪  ⚫  ✆  ✇  ✊  ✑  ✠  ✡  ✢  ✣  ✤  ✥  ✦  ✧  ❖  ❗  ❦  ❧  ➖  ➗  ➰  ➷  ➺  ➿  づ  ゞ  ガ  ギ  グ  ゲ  ゴ  ザ  ズ  ゼ  バ  ビ  ヷ  ヹ  ヺ  ㊊  ㊋  ㊌  ㊍  ㊎  ㊏  ㊐  유  ﹢  ﹣  ﹤  ﹥  🀄...
## Clase `POS` ##
(^L^)  (^‿^)  (°⌣°)  (•‿•)  (｡◕‿◕｡)  *-*  :)  :*  :-)  :-*  :-D  :3  :B  :D  :]  :p  ;)  ;-)  ;D  < (^^,) >  <3  <【☯】‿【☯】>  =)  =D  =^.^=  D:  Dx  XP  ^( \\\'‿\\\' )^  ^.^  ^^  ^_^  ^o^  ^‿^  n_n  q(❂‿❂)p  xD  {◕ ◡ ◕}  |◔◡◉|  ¢‿¢  Ü  ñ.ñ  ñ__ñ  ñ_ñ  ó‿ó  ت  ٩(-̮̮̃-̃)۶  ٩(-̮̮̃•̃)  ٩(^‿^)۶  ٩(͡๏̮͡๏)۶  ٩◔‿◔۶  ಠ◡ಠ  ༺‿༻  ღ  ᵔᴥᵔ  ‍👨❤‍️‍💋👨  ‍👩❤‍️‍💋👨  ‍👩❤‍️‍💋👩  †  ⇧  ⇪  √  ≧◡≦  ⊂◉‿◉つ  ⊙▃⊙  ⋆  ⍟  ┌(ಠ‿ಠ)┘  ╳  ●‿●  ◕‿◕٩(●̮̮̃•̃)۶  ◙‿◙  ☀  ☃  ★  ☆  ☋  ☑  ☕  ☘  ☚  ☜  ☥  ☦  ☮  ☯  ☺  ☻  ☼  ☽  ☾  ♉  ♔  ♕  ♖  ♚  ♛  ♡  ♣  ♥  ♥‿♥  ♧  ♩  ♪  ♫  ♬  ♻  ⚖  ⚛  ⛹  ⛹🏻  ⛹🏼  ⛹🏽  ⛹🏾  ⛹🏿  ✅  ✈  ✉  ✌  ✌🏻  ✌🏼  ✌🏽  ✌🏾  ✌🏿  ✓  ✔  ✙  ✚  ✛  ✜  ✝  ✞  ✟  ✨  ✩  ✪  ✫  ✬  ✭  ✮  ✯  ✰  ✱  ✲  ✳  ✴  ✵  ✶  ✷  ✸  ✹  ✺  ✻  ✼  ✽  ✾  ✿  ❀  ❀‿❀  ❁  ❂  ❃  ❇  ❈  ❉  ❊  ❋  ❕  ❣  ❤  ❤️  ❤️‿❤️  ❥  ➕  ➳  ➵  ➸  ➹  ➻  ➼  ➽  ⭐  シ  ジ  ゾ  ッ  ツ  ヅ  ㋡  乂◜◬◝乂  웃  🌄  🌅  🌷  🌹  🌺  🌻  🌼  🍀  🍻  🍾  🎂  🎄  🎆  🎇  🎈  🎉  🎊  🎑  🎖  🎟  🎧  🎫  🎵  🎶  🎼  🏅  🏆  🏩  🏵  🐭  🐮  🐱  🐵  👋  👋🏻  👋🏼  👋🏽  👋🏾  👋🏿  👌  👌🏻  👌🏼  👌🏽  👌🏾  👌🏿  👍  👍🏻  👍🏼  👍🏽  👍🏾  👍🏿  👎  👎🏻  👎🏼  👎🏽  👎🏾  👎🏿  👏...
## Clase `NEU` ##
#⃣  $_$  &  (-\\\'\\\'-)  (•̪●)  *⃣  -.-  -_-  ._.  0⃣  1⃣  2⃣  3⃣  4⃣  5⃣  6⃣  7⃣  8-|  8⃣  9⃣  :-|  >:-o  @_@  U_U  n__n  u.u  u_u  v( ‘.’ )v  {•̃̾_•̃̾}  |˚–˚|  ©  ¬¬  ®  ఠ_ఠ  ರ_ರ  ‍👨❤‍️👨  ‍👩❤‍️👨  ‍👩❤‍️👩  ™  ↔  ↕  ↖  ↗  ↘  ↙  ↩  ↪  ⇎_⇎  ⌚  ⌨  ⏩  ⏪  ⏫  ⏬  ⏭  ⏮  ⏯  ⏰  ⏱  ⏲  ⏳  ⏸  ⏹  ⏺  Ⓜ  ▪  ▫  ▶  ◀  ◐.̃◐  ◑.◑  ◔_◔  ◻  ◼  ◽  ◾  ☁  ☝🏻  ☝🏼  ☝🏽  ☝🏾  ☝🏿  ♀  ♁  ♂  ♈  ♗  ♘  ♙  ♜  ♝  ♞  ♟  ♠  ♢  ♦  ⚗  ⚙  ⚰  ⚱  ⚽  ⚾  ⛄  ⛅  ⛈  ⛎  ⛏  ⛑  ⛓  ⛩  ⛪  ⛰  ⛱  ⛲  ⛳  ⛴  ⛵  ⛷  ⛸  ⛺  ⛽  ✋🏻  ✋🏼  ✋🏽  ✋🏾  ✋🏿  ✍  ✍🏻  ✍🏼  ✍🏽  ✍🏾  ✍🏿  ✎  ✏  ✒  ❄  ❅  ❆  ❓  ❔  ➡  ⤴  ⤵  ⬅  ⬆  ⬇  ⬛  ⬜  ⭕  【•】_【•】  〰  〽  ㊗  ㊙  🃏  🅰  🅱  🅾  🅿  🆎  🆑  🆒  🆓  🆔  🆕  🆖  🆗  🆘  🆙  🆚  🇦🇨  🇦🇩  🇦🇪  🇦🇫  🇦🇬  🇦🇮  🇦🇱  🇦🇲  🇦🇴  🇦🇶  🇦🇷  🇦🇸  🇦🇹  🇦🇺  🇦🇼  🇦🇽  🇦🇿  🇧🇦  🇧🇧  🇧🇩  🇧🇪  🇧🇫  🇧🇬  🇧🇭  🇧🇮  🇧🇯  🇧🇱  🇧🇲  🇧🇳  🇧🇴  🇧🇶  🇧🇷  🇧🇸  🇧🇹  🇧🇻  🇧🇼  🇧🇾  🇧🇿  🇨🇦  🇨🇨  🇨🇩  🇨🇫  🇨🇬  🇨🇭  🇨🇮  🇨🇰  🇨🇱  🇨🇲  🇨🇳  🇨🇴  🇨🇵  🇨🇷  🇨🇺  🇨🇻  🇨🇼  🇨🇽  🇨🇾  🇨🇿  🇩🇪  🇩🇬  🇩🇯  🇩🇰  🇩🇲  🇩🇴  🇩🇿  🇪🇦  🇪🇨  🇪🇪  🇪🇬  🇪🇭  🇪🇷  🇪🇸  🇪🇹  🇪🇺  🇫🇮  🇫🇯  🇫🇰  🇫🇲  🇫🇴  🇫🇷  🇬🇦  🇬🇧  🇬🇩  🇬🇪  🇬🇫  🇬🇬  🇬🇭  🇬🇮  🇬🇱  🇬🇲  🇬🇳  🇬🇵 ...
## Clase `NEG` ##
'n'  (>.<)  (O.O)  (⊙̃.o ⊙.◎)  )-:  ):  ):-/  .l.  /:  :"(  :'(  :(  :-(  :-/  :-\  :-o  :-x  :/  :S  :[  :\  :c  :o  ='(  =(  =S  >.<  >:-(  >:o  ><  >_<  O.o  O.ó  TT  XC  X_X  \\\\m/(>.<)\\\\m/  o.Ó  õ.O  x_x  ¬ ¬  ¬ ¬*  ¬¬*  ».«  ò_ó  ॓_॔  ๏_๏  ⊝  ⌛  ⌤  ☂  ☒  ☔  ☟  ☠  ☢  ☣  ☹  ⚠  ⚡  ⛔  ✂  ✊🏻  ✊🏼  ✊🏽  ✊🏾  ✊🏿  ✋  ✕  ✖  ✗  ✘  ❌  ❎  〴⋋_⋌〵  卍  卐  🌁  🎗  👹  👺  👻  👽  👾  👿  💀  💔  💢  💣  💦  💨  💩  💫  📉  🖕  🖕🏻  🖕🏼  🖕🏽  🖕🏾  🖕🏿  🗯  😓  😔  😖  😞  😟  😠  😡  😢  😣  😥  😦  😧  😨  😩  😪  😫  😬  😭  😮  😯  😰  😱  😲  😳  😵  😷  😾  😿  🙀  🙁  🙃  🙅  🙅🏻  🙅🏼  🙅🏽  🙅🏾  🙅🏿  🙈  🙉  🙊  🙎  🙎🏻  🙎🏼  🙎🏽  🙎🏾  🙎🏿  🚫  🚳  🤒  🤕  🤘  🤘🏻  🤘🏼  🤘🏽  🤘🏾  🤘🏿...

In [7]:
from itertools import combinations
text = "qué buena esta la plática"
tokenizers = [1, 2, 3, -1, -2, (2,1), (2,2)]

num = 1
output = ["""

## Tokenizadores Ejemplos ##
La codificación es:

- entero positivo: q-grama de caracter
- entero engativo: w-word (n-grama de palabra)
- tupla: skip-gram (tamaño, salto)

"""]
for ltokens in range(len(tokenizers)):
    output.append('## Combinaciones de tamaño {0} ##'.format(ltokens+1))
    output.append('|id|combinación|tokens|')
    output.append('|--|-----------|------|')
    for comb in combinations(tokenizers, ltokens+1):
        model = TextModel([], token_list=comb)
        output.append("|{0}|{1}|{2}|".format(num, comb, ", ".join(model.tokenize(text))))
        num += 1

Markdown("\n".join(output))



## Tokenizadores Ejemplos ##
La codificación es:

- entero positivo: q-grama de caracter
- entero engativo: w-word (n-grama de palabra)
- tupla: skip-gram (tamaño, salto)


## Combinaciones de tamaño 1 ##
|id|combinación|tokens|
|--|-----------|------|
|1|(1,)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~|
|2|(2,)|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~|
|3|(3,)|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~|
|4|(-1,)|que, buena, esta, la, platica|
|5|(-2,)|que~buena, buena~esta, esta~la, la~platica|
|6|((2, 1),)|que~esta, buena~la, esta~platica|
|7|((2, 2),)|que~la, buena~platica|
## Combinaciones de tamaño 2 ##
|id|combinación|tokens|
|--|-----------|------|
|8|(1, 2)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~|
|9|(1, 3)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~|
|10|(1, -1)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que, buena, esta, la, platica|
|11|(1, -2)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que~buena, buena~esta, esta~la, la~platica|
|12|(1, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que~esta, buena~la, esta~platica|
|13|(1, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que~la, buena~platica|
|14|(2, 3)|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~|
|15|(2, -1)|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica|
|16|(2, -2)|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~buena, buena~esta, esta~la, la~platica|
|17|(2, (2, 1))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~esta, buena~la, esta~platica|
|18|(2, (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~la, buena~platica|
|19|(3, -1)|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica|
|20|(3, -2)|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica|
|21|(3, (2, 1))|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~esta, buena~la, esta~platica|
|22|(3, (2, 2))|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~la, buena~platica|
|23|(-1, -2)|que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica|
|24|(-1, (2, 1))|que, buena, esta, la, platica, que~esta, buena~la, esta~platica|
|25|(-1, (2, 2))|que, buena, esta, la, platica, que~la, buena~platica|
|26|(-2, (2, 1))|que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|27|(-2, (2, 2))|que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|28|((2, 1), (2, 2))|que~esta, buena~la, esta~platica, que~la, buena~platica|
## Combinaciones de tamaño 3 ##
|id|combinación|tokens|
|--|-----------|------|
|29|(1, 2, 3)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~|
|30|(1, 2, -1)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica|
|31|(1, 2, -2)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~buena, buena~esta, esta~la, la~platica|
|32|(1, 2, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~esta, buena~la, esta~platica|
|33|(1, 2, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~la, buena~platica|
|34|(1, 3, -1)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica|
|35|(1, 3, -2)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica|
|36|(1, 3, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~esta, buena~la, esta~platica|
|37|(1, 3, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~la, buena~platica|
|38|(1, -1, -2)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica|
|39|(1, -1, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica|
|40|(1, -1, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que, buena, esta, la, platica, que~la, buena~platica|
|41|(1, -2, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|42|(1, -2, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|43|(1, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que~esta, buena~la, esta~platica, que~la, buena~platica|
|44|(2, 3, -1)|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica|
|45|(2, 3, -2)|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica|
|46|(2, 3, (2, 1))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~esta, buena~la, esta~platica|
|47|(2, 3, (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~la, buena~platica|
|48|(2, -1, -2)|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica|
|49|(2, -1, (2, 1))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica|
|50|(2, -1, (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~la, buena~platica|
|51|(2, -2, (2, 1))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|52|(2, -2, (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|53|(2, (2, 1), (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~esta, buena~la, esta~platica, que~la, buena~platica|
|54|(3, -1, -2)|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica|
|55|(3, -1, (2, 1))|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica|
|56|(3, -1, (2, 2))|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~la, buena~platica|
|57|(3, -2, (2, 1))|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|58|(3, -2, (2, 2))|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|59|(3, (2, 1), (2, 2))|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~esta, buena~la, esta~platica, que~la, buena~platica|
|60|(-1, -2, (2, 1))|que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|61|(-1, -2, (2, 2))|que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|62|(-1, (2, 1), (2, 2))|que, buena, esta, la, platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|63|(-2, (2, 1), (2, 2))|que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
## Combinaciones de tamaño 4 ##
|id|combinación|tokens|
|--|-----------|------|
|64|(1, 2, 3, -1)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica|
|65|(1, 2, 3, -2)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica|
|66|(1, 2, 3, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~esta, buena~la, esta~platica|
|67|(1, 2, 3, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~la, buena~platica|
|68|(1, 2, -1, -2)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica|
|69|(1, 2, -1, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica|
|70|(1, 2, -1, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~la, buena~platica|
|71|(1, 2, -2, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|72|(1, 2, -2, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|73|(1, 2, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~esta, buena~la, esta~platica, que~la, buena~platica|
|74|(1, 3, -1, -2)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica|
|75|(1, 3, -1, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica|
|76|(1, 3, -1, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~la, buena~platica|
|77|(1, 3, -2, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|78|(1, 3, -2, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|79|(1, 3, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~esta, buena~la, esta~platica, que~la, buena~platica|
|80|(1, -1, -2, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|81|(1, -1, -2, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|82|(1, -1, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|83|(1, -2, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|84|(2, 3, -1, -2)|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica|
|85|(2, 3, -1, (2, 1))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica|
|86|(2, 3, -1, (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~la, buena~platica|
|87|(2, 3, -2, (2, 1))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|88|(2, 3, -2, (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|89|(2, 3, (2, 1), (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~esta, buena~la, esta~platica, que~la, buena~platica|
|90|(2, -1, -2, (2, 1))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|91|(2, -1, -2, (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|92|(2, -1, (2, 1), (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|93|(2, -2, (2, 1), (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|94|(3, -1, -2, (2, 1))|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|95|(3, -1, -2, (2, 2))|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|96|(3, -1, (2, 1), (2, 2))|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|97|(3, -2, (2, 1), (2, 2))|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|98|(-1, -2, (2, 1), (2, 2))|que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
## Combinaciones de tamaño 5 ##
|id|combinación|tokens|
|--|-----------|------|
|99|(1, 2, 3, -1, -2)|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica|
|100|(1, 2, 3, -1, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica|
|101|(1, 2, 3, -1, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~la, buena~platica|
|102|(1, 2, 3, -2, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|103|(1, 2, 3, -2, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|104|(1, 2, 3, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~esta, buena~la, esta~platica, que~la, buena~platica|
|105|(1, 2, -1, -2, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|106|(1, 2, -1, -2, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|107|(1, 2, -1, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|108|(1, 2, -2, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|109|(1, 3, -1, -2, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|110|(1, 3, -1, -2, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|111|(1, 3, -1, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|112|(1, 3, -2, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|113|(1, -1, -2, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|114|(2, 3, -1, -2, (2, 1))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|115|(2, 3, -1, -2, (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|116|(2, 3, -1, (2, 1), (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|117|(2, 3, -2, (2, 1), (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|118|(2, -1, -2, (2, 1), (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|119|(3, -1, -2, (2, 1), (2, 2))|~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
## Combinaciones de tamaño 6 ##
|id|combinación|tokens|
|--|-----------|------|
|120|(1, 2, 3, -1, -2, (2, 1))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica|
|121|(1, 2, 3, -1, -2, (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~la, buena~platica|
|122|(1, 2, 3, -1, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|123|(1, 2, 3, -2, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|124|(1, 2, -1, -2, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|125|(1, 3, -1, -2, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
|126|(2, 3, -1, -2, (2, 1), (2, 2))|~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|
## Combinaciones de tamaño 7 ##
|id|combinación|tokens|
|--|-----------|------|
|127|(1, 2, 3, -1, -2, (2, 1), (2, 2))|~, q, u, e, ~, b, u, e, n, a, ~, e, s, t, a, ~, l, a, ~, p, l, a, t, i, c, a, ~, ~q, qu, ue, e~, ~b, bu, ue, en, na, a~, ~e, es, st, ta, a~, ~l, la, a~, ~p, pl, la, at, ti, ic, ca, a~, ~qu, que, ue~, e~b, ~bu, bue, uen, ena, na~, a~e, ~es, est, sta, ta~, a~l, ~la, la~, a~p, ~pl, pla, lat, ati, tic, ica, ca~, que, buena, esta, la, platica, que~buena, buena~esta, esta~la, la~platica, que~esta, buena~la, esta~platica, que~la, buena~platica|